In [ ]:
import pandas as pd
import os
import shutil

In [ ]:
# Data of +ve patients from github
# Link: https://github.com/ieee8023/covid-chestxray-dataset

File_path = 'Covid_Chestxray_Dataset/metadata.csv'
Image_path = 'Covid_Chestxray_Dataset/images'

In [ ]:
df = pd.read_csv(File_path)
print(df.shape)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
print(df['view'])

In [ ]:
Target_dir = 'Covid_Dataset/Covid'
0
if not os.path.exists(Target_dir):
    os.mkdir(Target_dir)

In [ ]:
count = 0

for (i,row) in df.iterrows():
    if row['finding'] == 'Pneumonia/Viral/COVID-19' and row['view'] == 'PA':
        filename = row['filename']
        image_path = os.path.join(Image_path,filename)
        image_copy_path = os.path.join(Target_dir,filename)
        shutil.copy2(image_path,image_copy_path)
        print("Moving Image",count)
        count += 1

print(count)

In [ ]:

# Data of normal patients from kaggle
# Link: https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia

import random
Kaggle_File = 'NORMAL_Xray'
Normal_dir = 'Covid_Dataset/Normal'

if not os.path.exists(Normal_dir):
    os.mkdir(Normal_dir)
image_names = os.listdir(Kaggle_File)
random.shuffle(image_names)

In [ ]:
for i in range(196):
    image_name = image_names[i]
    image_path = os.path.join(Kaggle_File,image_name)
    
    target_path = os.path.join(Normal_dir,image_name)
    shutil.copy2(image_path,target_path)
    print('Moving Image',i)

In [ ]:
# For Train And Validation Dataset
SPLIT = 0.8
for f in os.listdir('Covid_Dataset/Train'):
    path = "Covid_Dataset/Train/"+f
    imgs = os.listdir(path)
    
    split_size = int(SPLIT*len(imgs))
    files_to_move = imgs[split_size:]
    print(len(files_to_move))
    
    for val_img  in files_to_move:
        src = os.path.join(path,val_img)
        dest = os.path.join('Covid_Dataset/Valid/'+f,val_img)
        shutil.move(src,dest)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import * 
from keras.preprocessing import image

In [2]:
# CNN Based Model in Keras

model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 64)        0

In [5]:
# Train from scratch
train_datagen = image.ImageDataGenerator(rescale = 1./255,
                                            shear_range = 0.2,
                                            zoom_range = 0.2,
                                            horizontal_flip = True)

test_dataset = image.ImageDataGenerator(rescale=1./255,
                                       shear_range = 0.2,
                                            zoom_range = 0.2,
                                            horizontal_flip = True)

In [6]:
train_generator = train_datagen.flow_from_directory('Covid_Dataset/Train',
                                                    target_size = (224,224),
                                                    batch_size = 32,
                                                    class_mode = 'binary')

Found 312 images belonging to 2 classes.


In [7]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [8]:
validation_generator = test_dataset.flow_from_directory('Covid_Dataset/Valid',
                                                    target_size = (224,224),
                                                    batch_size = 32,
                                                    class_mode = 'binary')

Found 80 images belonging to 3 classes.


In [9]:
hist = model.fit_generator(train_generator,
                            steps_per_epoch=8,
                            epochs = 10,
                            validation_data = validation_generator,
                            validation_steps=2)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
8/8 [==============================] - 29s 4s/step - loss: 1.3488 - accuracy: 0.5645 - val_loss: 0.7567 - val_accuracy: 0.0000e+00
Epoch 2/10
8/8 [==============================] - 28s 3s/step - loss: 0.6172 - accuracy: 0.6613 - val_loss: 0.5990 - val_accuracy: 0.0781
Epoch 3/10
8/8 [==============================] - 30s 4s/step - loss: 0.4350 - accuracy: 0.8065 - val_loss: 0.4292 - val_accuracy: 0.0000e+00
Epoch 4/10
8/8 [==============================] - 28s 4s/step - loss: 0.4005 - accuracy: 0.8347 - val_loss: 0.1000 - val_accuracy: 0.0781
Epoch 5/10
8/8 [==============================] - 30s 4s/step - loss: 0.3122 - accuracy: 0.8952 - val_loss: 0.2124 - val_accuracy: 0.0938
Epoch 6/10
8/8 [==============================] - 30s 4s/step - loss: 0.2509 - accuracy: 0.9141 - val_loss: -0.6589 - val_accuracy: 0.0781
Epoch 7/10
8/8 [==============================] - 30s 4s/step - loss: 0.1885 - accuracy

In [13]:
filenames = validation_generator.filenames
nb_samples = len(filenames)

predict = model.predict_generator(validation_generator,steps=np.ceil(nb_samples/32))

In [10]:
import anvil.server
anvil.server.connect("3ARPWPQT5MDDTIG5MGM2DRB4-Q6B6LPKYKBQ44QVX")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


In [14]:
import anvil.media

@anvil.server.callable
def classify_image(file):
    with anvil.media.TempFile(file) as filename:
        img = load_img(filename)
    img = img.resize((128, 128), resample=PIL.Image.BICUBIC)
    arr = img_to_array(img)
    arr = np.expand_dims(arr, axis=0)
    arr /= 255
    score = model.predict(arr)
    return ('Covid Negative' if score < 0.5 else 'Covid Positive', float(score))